# Happy or not?

I'll be using transfer learning with keras, to find out if a certain image depicts a happy person or not. There might be better or more efficient approaches to doing this, and maybe this turns out to be overkill. However, I wish to know if this would work well. 
I'll be using pre-trained inception model v3 and add a couple of FC layers at the end to check how well it does. 
For this I have a dataset of about 250 images scrapped off the internet. I would suggest you to not use google since it is biased in the sense you would not get asians or indians pictures in the pool. Bing is better. Best would be instagram. 

In [2]:
import numpy as np
import pandas as pd
import h5py

import os, cv2, random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline 

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalAveragePooling2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

# First attempt will be with the inception v3 model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K

In [16]:
TRAIN_DIR = './dataset/'
CLASSES = ['happy', 'nothappy']

# Image dimentions for resizing:
ROWS = 299  # Default input of inception v3 
COLS = 299 # Default input of inception v3 
CHANNELS = 3

In [17]:
def get_images(emotion):
    """Load files from train folder"""
    emotion_dir = TRAIN_DIR+'{}'.format(emotion)
    images = [emotion+'/'+im for im in os.listdir(emotion_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (COLS, ROWS), interpolation=cv2.INTER_CUBIC)
    return im


files = []
y_all = []

for emotion in CLASSES:
    emotion_files = get_images(emotion)
    files.extend(emotion_files)
    
    y_emotion = np.tile(emotion, len(emotion_files))
    y_all.extend(y_emotion)
    print("{0} photos of {1}".format(len(emotion_files), emotion))
    
y_all = np.array(y_all)

256 photos of happy
190 photos of nothappy


In [18]:
X_all = np.ndarray((len(files), ROWS, COLS, CHANNELS), dtype=np.uint8)

In [19]:
for i, im in enumerate(files): 
    X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

Processed 0 of 446
(446, 299, 299, 3)


In [11]:
y_all

array(['happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'hap

In [20]:
y_all = LabelEncoder().fit_transform(y_all)
y_all

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

In [21]:
y_all = np_utils.to_categorical(y_all)
y_all

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, 
                                                    test_size=0.25, random_state=23, 
                                                    stratify=y_all)

In [23]:
X_train.shape

(334, 299, 299, 3)

In [24]:
y_train.shape

(334, 2)

In [31]:
y_train

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,

In [32]:
X_valid.shape

(112, 299, 299, 3)

In [25]:
# Remember that as a part of the preprocessing we have already scaled down the images to fit
# the inception model input requirements.

# create the base pre-trained model from Keras library
base_model = InceptionV3(weights='imagenet', include_top=False) #change weights to 'imagenet' on your local build

In [49]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer for the FISH_CLASSES = 8 we are trying to predict
predictions = Dense(len(CLASSES), activation='sigmoid', name='prediction_layer')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [50]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first N layers and unfreeze the rest:

N=172

for layer in model.layers[:N]:
    layer.trainable = False
for layer in model.layers[N:]:
    layer.trainable = True

In [51]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=5,
              validation_split=0.25, verbose=1, shuffle=True)

Train on 250 samples, validate on 84 samples
Epoch 1/5
250/250 [==============================] - 244s 975ms/step - loss: 0.6838 - val_loss: 6.9286
Epoch 2/5
250/250 [==============================] - 252s 1s/step - loss: 0.6832 - val_loss: 6.9070
Epoch 3/5
